# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/venv/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client = Client('dask-scheduler:8786')
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

/venv/lib/python3.8/site-packages/distributed/client.py:1135: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| blosc   | None          | 1.9.2         | 1.9.2         |
| lz4     | None          | 3.0.2         | 3.0.2         |
| msgpack | 1.0.2         | 1.0.0         | 1.0.0         |
| numpy   | 1.20.1        | 1.19.4        | 1.19.4        |
| python  | 3.8.8.final.0 | 3.8.0.final.0 | 3.8.0.final.0 |
+---------+---------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tcp://dask-scheduler:8786 Dashboard: http://dask-scheduler:8787/status,Cluster Workers: 4 Cores: 4 Memory: 1.60 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
df.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1044,Dan,0.020118,-0.148869
2000-01-01 00:00:01,1066,Zelda,0.781411,-0.502504
2000-01-01 00:00:02,1029,George,-0.486709,-0.719912
2000-01-01 00:00:03,981,Zelda,-0.037930,0.037740
2000-01-01 00:00:04,1010,Ingrid,0.514863,0.993939


In [4]:
import os
import datetime

if not os.path.exists('../data'):
    os.mkdir('../data')

def name(i):
    """ Provide date for filename given index

    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))

df.to_csv('../data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [8]:
!ls ../data/*.csv | head

../data/2000-01-01.csv
../data/2000-01-02.csv
../data/2000-01-03.csv
../data/2000-01-04.csv
../data/2000-01-05.csv
../data/2000-01-06.csv
../data/2000-01-07.csv
../data/2000-01-08.csv
../data/2000-01-09.csv
../data/2000-01-10.csv


In [9]:
!head ../data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1044,Dan,0.020117934837050377,-0.1488688104377689
2000-01-01 00:00:01,1066,Zelda,0.7814111633789307,-0.5025042497587033
2000-01-01 00:00:02,1029,George,-0.48670862324897235,-0.7199121239449857
2000-01-01 00:00:03,981,Zelda,-0.03792980809615498,0.03773971439975732
2000-01-01 00:00:04,1010,Ingrid,0.514863201056921,0.9939394212728145
2000-01-01 00:00:05,967,George,-0.21685000268849808,0.1268974661961928
2000-01-01 00:00:06,960,Zelda,0.3004966025460807,0.7072398572969063
2000-01-01 00:00:07,986,Jerry,0.5523556278906749,-0.20959456013132827
2000-01-01 00:00:08,1036,Patricia,-0.1982083694650214,-0.6558886240197199


In [10]:
!head ../data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1046,Frank,-0.027477835096968883,-0.8660928407996125
2000-01-30 00:00:01,993,Michael,-0.8323542156398003,0.3194860371937265
2000-01-30 00:00:02,1012,Jerry,0.5415565579636861,0.9321589729653199
2000-01-30 00:00:03,1021,Sarah,-0.6454106024041866,-0.5846859974520537
2000-01-30 00:00:04,976,Laura,0.4552121863025482,0.9592600686692092
2000-01-30 00:00:05,971,Dan,-0.46938924478939326,0.9205347299685607
2000-01-30 00:00:06,994,Tim,-0.5436660384405145,-0.2923311146144161
2000-01-30 00:00:07,972,Dan,-0.9603243986853793,0.8131546536493
2000-01-30 00:00:08,1049,Quinn,-0.705580416696681,-0.261972022905381


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [11]:
import pandas as pd

df = pd.read_csv('../data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1044,Dan,0.020118,-0.148869
1,2000-01-01 00:00:01,1066,Zelda,0.781411,-0.502504
2,2000-01-01 00:00:02,1029,George,-0.486709,-0.719912
3,2000-01-01 00:00:03,981,Zelda,-0.037930,0.037740
4,2000-01-01 00:00:04,1010,Ingrid,0.514863,0.993939


In [12]:
import dask.dataframe as dd

df = dd.read_csv('../data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [13]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1044,Dan,0.020118,-0.148869
1,2000-01-01 00:00:01,1066,Zelda,0.781411,-0.502504
2,2000-01-01 00:00:02,1029,George,-0.486709,-0.719912
3,2000-01-01 00:00:03,981,Zelda,-0.037930,0.037740
4,2000-01-01 00:00:04,1010,Ingrid,0.514863,0.993939


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [12]:
pd.read_csv?

<function pandas.io.parsers.read_csv(filepath_or_buffer: Union[ForwardRef('PathLike[str]'), str, IO[~T], io.RawIOBase, io.BufferedIOBase, io.TextIOBase, _io.TextIOWrapper, mmap.mmap], sep=<object object at 0x7fc71cea05c0>, delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal: str = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=Tru

In [13]:
dd.read_csv?

<function dask.dataframe.io.csv.make_reader.<locals>.read(urlpath, blocksize='default', lineterminator=None, compression='infer', sample=256000, enforce=False, assume_missing=False, storage_options=None, include_path_column=False, **kwargs)>

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [14]:
df = dd.read_csv('../data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [15]:
%time df.groupby('name').x.mean().compute()

CPU times: user 65.7 ms, sys: 7.2 ms, total: 72.9 ms
Wall time: 3.45 s


name
Alice      -0.000377
Bob         0.000254
Charlie     0.000290
Dan         0.000659
Edith      -0.000065
Frank      -0.003392
George     -0.000688
Hannah     -0.001032
Ingrid     -0.000485
Jerry      -0.000988
Kevin      -0.000107
Laura       0.002847
Michael    -0.001403
Norbert     0.002549
Oliver     -0.002447
Patricia   -0.001809
Quinn       0.003286
Ray        -0.000691
Sarah       0.001424
Tim        -0.000972
Ursula     -0.001154
Victor      0.001338
Wendy       0.000441
Xavier     -0.000208
Yvonne      0.000550
Zelda      -0.003028
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [16]:
df.to_parquet('../data/2000-01.parquet', engine='pyarrow')

In [17]:
!ls ../data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [18]:
df = dd.read_parquet('../data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [19]:
%time df.groupby('name').x.mean().compute()

CPU times: user 53.6 ms, sys: 6.71 ms, total: 60.3 ms
Wall time: 986 ms


name
Alice      -0.000377
Bob         0.000254
Charlie     0.000290
Dan         0.000659
Edith      -0.000065
Frank      -0.003392
George     -0.000688
Hannah     -0.001032
Ingrid     -0.000485
Jerry      -0.000988
Kevin      -0.000107
Laura       0.002847
Michael    -0.001403
Norbert     0.002549
Oliver     -0.002447
Patricia   -0.001809
Quinn       0.003286
Ray        -0.000691
Sarah       0.001424
Tim        -0.000972
Ursula     -0.001154
Victor      0.001338
Wendy       0.000441
Xavier     -0.000208
Yvonne      0.000550
Zelda      -0.003028
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [20]:
%%time
df = dd.read_parquet('../data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 95.4 ms, sys: 69.5 ms, total: 165 ms
Wall time: 1.41 s


name
Alice      -0.000377
Bob         0.000254
Charlie     0.000290
Dan         0.000659
Edith      -0.000065
Frank      -0.003392
George     -0.000688
Hannah     -0.001032
Ingrid     -0.000485
Jerry      -0.000988
Kevin      -0.000107
Laura       0.002847
Michael    -0.001403
Norbert     0.002549
Oliver     -0.002447
Patricia   -0.001809
Quinn       0.003286
Ray        -0.000691
Sarah       0.001424
Tim        -0.000972
Ursula     -0.001154
Victor      0.001338
Wendy       0.000441
Xavier     -0.000208
Yvonne      0.000550
Zelda      -0.003028
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html